In [19]:
from sqlalchemy import create_engine
import pandas as pd
from time import time

In [28]:
user = "root"
password = "root"
host = "localhost"
port = 5432
db = "ny_taxi"
url = f"postgresql://{user}:{password}@{host}:{port}/{db}"
engine = create_engine(url)

In [21]:
df_iter = pd.read_csv('yellow_tripdata_2021-01.csv', iterator=True, chunksize=100000)

In [22]:
df = next(df_iter)
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2021-01-01 00:30:10,2021-01-01 00:36:12,1,2.10,1,N,142,43,2,8.0,3.0,0.5,0.00,0.0,0.3,11.80,2.5
1,1,2021-01-01 00:51:20,2021-01-01 00:52:19,1,0.20,1,N,238,151,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
2,1,2021-01-01 00:43:30,2021-01-01 01:11:06,1,14.70,1,N,132,165,1,42.0,0.5,0.5,8.65,0.0,0.3,51.95,0.0
3,1,2021-01-01 00:15:48,2021-01-01 00:31:01,0,10.60,1,N,138,132,1,29.0,0.5,0.5,6.05,0.0,0.3,36.35,0.0
4,2,2021-01-01 00:31:49,2021-01-01 00:48:21,1,4.94,1,N,68,33,1,16.5,0.5,0.5,4.06,0.0,0.3,24.36,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2021-01-04 14:04:31,2021-01-04 14:08:52,3,0.70,1,N,234,224,2,5.0,2.5,0.5,0.00,0.0,0.3,8.30,2.5
99996,1,2021-01-04 14:18:46,2021-01-04 14:35:45,2,3.30,1,N,234,236,1,14.5,2.5,0.5,3.55,0.0,0.3,21.35,2.5
99997,1,2021-01-04 14:42:41,2021-01-04 14:59:22,2,4.70,1,N,236,79,1,17.0,2.5,0.5,4.05,0.0,0.3,24.35,2.5
99998,2,2021-01-04 14:39:02,2021-01-04 15:09:37,2,17.95,2,N,132,148,1,52.0,0.0,0.5,5.00,0.0,0.3,60.30,2.5


In [23]:
print(pd.io.sql.get_schema(df, 'yellow_taxi'))

CREATE TABLE "yellow_taxi" (
"VendorID" INTEGER,
  "tpep_pickup_datetime" TEXT,
  "tpep_dropoff_datetime" TEXT,
  "passenger_count" INTEGER,
  "trip_distance" REAL,
  "RatecodeID" INTEGER,
  "store_and_fwd_flag" TEXT,
  "PULocationID" INTEGER,
  "DOLocationID" INTEGER,
  "payment_type" INTEGER,
  "fare_amount" REAL,
  "extra" REAL,
  "mta_tax" REAL,
  "tip_amount" REAL,
  "tolls_amount" REAL,
  "improvement_surcharge" REAL,
  "total_amount" REAL,
  "congestion_surcharge" REAL
)


In [24]:
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

In [25]:
print(pd.io.sql.get_schema(df, 'yellow_taxi'))

CREATE TABLE "yellow_taxi" (
"VendorID" INTEGER,
  "tpep_pickup_datetime" TIMESTAMP,
  "tpep_dropoff_datetime" TIMESTAMP,
  "passenger_count" INTEGER,
  "trip_distance" REAL,
  "RatecodeID" INTEGER,
  "store_and_fwd_flag" TEXT,
  "PULocationID" INTEGER,
  "DOLocationID" INTEGER,
  "payment_type" INTEGER,
  "fare_amount" REAL,
  "extra" REAL,
  "mta_tax" REAL,
  "tip_amount" REAL,
  "tolls_amount" REAL,
  "improvement_surcharge" REAL,
  "total_amount" REAL,
  "congestion_surcharge" REAL
)


In [26]:
df.head(n=0).to_sql(name='yellow_taxi', con=engine, if_exists='replace')

In [29]:
%time df.to_sql(name='yellow_taxi', con=engine, if_exists='append')

CPU times: user 11.6 s, sys: 4.63 s, total: 16.2 s
Wall time: 1min 23s


In [30]:
while True:
    t_start = time()
    df = next(df_iter)

    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

    df.to_sql(name='yellow_taxi', con=engine, if_exists='append')

    t_end = time()
    print("inserted chunk... took %.3f second" % (t_end-t_start))    

inserted chunk... took 75.131 second
inserted chunk... took 76.554 second
inserted chunk... took 76.181 second
inserted chunk... took 77.019 second
inserted chunk... took 75.795 second
inserted chunk... took 78.697 second
inserted chunk... took 79.127 second
inserted chunk... took 79.484 second
inserted chunk... took 82.837 second
inserted chunk... took 81.168 second
inserted chunk... took 82.800 second


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3191: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


inserted chunk... took 84.852 second
inserted chunk... took 59.100 second


StopIteration: 